# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [4]:
sales.column_names

<bound method SFrame.column_names of Columns:
	id	str
	date	datetime
	price	float
	bedrooms	float
	bathrooms	float
	sqft_living	float
	sqft_lot	float
	floors	float
	waterfront	int
	view	int
	condition	int
	grade	float
	sqft_above	float
	sqft_basement	float
	yr_built	float
	yr_renovated	float
	zipcode	str
	lat	float
	long	float
	sqft_living15	float
	sqft_lot15	float
	sqft_living_sqrt	float
	sqft_lot_sqrt	float
	bedrooms_square	float
	floors_square	float

Rows: 21613

Data:
+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 7129300520 | 2014-10-13 00:00:00+00:00 |  221900.0 |   3.0    |    1.0    |
| 6414100192 | 2014-12-09 00:00:00+00:00 |  538000.0 |   3.0    |    2.25   |
| 5631500400 | 2015-02-25 00:00:00+00:00 |  180000.0 |   2.0    |    1.0    |
| 2487200875 | 2014-12-09 00:00:00+00:00 |  604000.0 | 

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.349926     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.453423     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [7]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print ('Learned polynomial for degree ' + str(deg) + ':')
    w.reverse()
    print (np.poly1d(w))

Find what features had non-zero weight.

In [25]:
import numpy as np
print(model_all.coefficients[model_all.coefficients['value'] != 0])
print_coefficients(model_all)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.0559504957  |  None  |
|    bathrooms     |  None | 8468.531086910072  |  None  |
|   sqft_living    |  None | 24.420720982445214 |  None  |
| sqft_living_sqrt |  None | 350.0605533860648  |  None  |
|      grade       |  None | 842.0680348976282  |  None  |
|    sqft_above    |  None | 20.024722417091304 |  None  |
+------------------+-------+--------------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

Ans: bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation_set, testing_set) = sales.random_split(.9,seed=1) # initial train/test split
(training_set, validation_set) = training_and_validation_set.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [10]:
def get_best_l1_from_lasso(data, lasso_features, l1_penalty_list, validation_data):
    l1_penalty_list = np.logspace(1, 7, 13)
    best_l1_penalty = None
    min_RSS = None
    non_zero_weights = None
    RSS_list = list()
    for penalty in l1_penalty_list:
        model = turicreate.linear_regression.create(data, target= 'price', features= lasso_features, l1_penalty= penalty, l2_penalty= 0.0, validation_set= None, verbose= False)
        RSS = ((model.predict(validation_data) - validation_data['price'])**2).sum()
        if min_RSS == None:
            min_RSS = RSS
            best_l1_penalty = penalty
            non_zero_weights = model.coefficients['value'].nnz()
        if RSS < min_RSS:
            min_RSS = RSS
            best_l1_penalty = penalty
            non_zero_weights = model.coefficients['value'].nnz()
        RSS_list.append(RSS)
    return best_l1_penalty, non_zero_weights,RSS_list

In [11]:
l1_penalty_list = np.logspace(1, 7, num=13)
(best_l1_penalty, potential_weights, RSS_list) = get_best_l1_from_lasso(training_set, all_features, l1_penalty_list, validation_set)

In [12]:
for i in range(len(l1_penalty_list)):
    print('l1 penalty = ',l1_penalty_list[i], '\t RSS= ', RSS_list[i])

l1 penalty =  10.0 	 RSS=  625766285142459.9
l1 penalty =  31.622776601683793 	 RSS=  625766285362394.1
l1 penalty =  100.0 	 RSS=  625766286057885.0
l1 penalty =  316.22776601683796 	 RSS=  625766288257224.8
l1 penalty =  1000.0 	 RSS=  625766295212186.6
l1 penalty =  3162.2776601683795 	 RSS=  625766317206080.5
l1 penalty =  10000.0 	 RSS=  625766386760658.1
l1 penalty =  31622.776601683792 	 RSS=  625766606749278.5
l1 penalty =  100000.0 	 RSS=  625767302791634.1
l1 penalty =  316227.7660168379 	 RSS=  625769507643886.2
l1 penalty =  1000000.0 	 RSS=  625776517727024.0
l1 penalty =  3162277.6601683795 	 RSS=  625799062845467.0
l1 penalty =  10000000.0 	 RSS=  625883719085425.2


In [13]:
best_l1_penalty

10.0

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

Ans: 10.0

In [14]:
potential_weights

18

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

Ans: 18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [15]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [16]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [17]:
def get_lasso_sparsity(data, lasso_features, l1_penalty_list, validation_data):
    best_l1_penalty = None
    min_RSS = None
    best_non_zero_weights = None
    non_zeros_list = list()
    model_list = list()
    RSS_list = list()
    for penalty in l1_penalty_list:
        model = turicreate.linear_regression.create(data, target= 'price', features= lasso_features, l1_penalty= penalty, l2_penalty= 0.0, validation_set= None, verbose= False)
        RSS = ((model.predict(validation_data) - validation_data['price'])**2).sum()
        if min_RSS == None:
            min_RSS = RSS
            best_l1_penalty = penalty
            non_zero_weights = model.coefficients['value'].nnz()
        if RSS < min_RSS:
            min_RSS = RSS
            best_l1_penalty = penalty
            non_zero_weights = model.coefficients['value'].nnz()
        RSS_list.append(RSS)
        non_zeros_list.append(model.coefficients['value'].nnz())
        model_list.append(model)
    return best_l1_penalty, non_zero_weights,RSS_list, model_list, non_zeros_list

In [18]:
(best_l1_penalty, potential_weights, RSS_list,model_list, non_zeros_list) = get_lasso_sparsity(training_set,  all_features, l1_penalty_values, validation_set)
print(best_l1_penalty)
print(potential_weights)
print(non_zeros_list)
# print(RSS_list)
# print(model_list)

100000000.0
18
[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [19]:
print(non_zeros_list)

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


In [20]:
l1_penalty_min = l1_penalty_values[14]
l1_penalty_max = l1_penalty_values[15]
print(l1_penalty_min)
print(l1_penalty_max)

2976351441.6313133
3792690190.7322536


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

Ans: l1_penalty_min = 2976351441.6313133, l1_penalty_max= 3792690190.7322536

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [21]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [22]:
(best_l1_penalty, potential_weights, RSS_list,model_list, non_zeros_list) = get_lasso_sparsity(training_set,  all_features, l1_penalty_values, validation_set)

In [26]:
print(best_l1_penalty)
print(potential_weights)
print(non_zeros_list)
print(RSS_list[11:15])
print(l1_penalty_values[11:15])

2976351441.6313133
10
[10, 10, 10, 10, 10, 10, 10, 10, 8, 8, 8, 7, 7, 7, 7, 6, 6, 6, 6, 6]
[1046937488751711.1, 1051147625612860.9, 1055992735342999.1, 1060799531763287.9]
[3.44896861e+09 3.49193381e+09 3.53489901e+09 3.57786420e+09]


In [24]:
model_features = model_list[11].coefficients
model_features = model_features[model_features['value'] != 0]
model_features

name,index,value,stderr
(intercept),None,222253.19254432785,None
bedrooms,None,661.7227177822587,None
bathrooms,None,15873.957259267981,None
sqft_living,None,32.41022145125964,None
sqft_living_sqrt,None,690.1147733133256,None
grade,None,2899.4202697498786,None
sqft_above,None,30.011575302201045,None


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?

Ans: l1_penalty= 3.44896861e+09
2. What features in this model have non-zero coefficients?

Ans: bedrooms, bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above